In [1]:
# pip install python_speech_features

In [2]:
import numpy as np
import scipy.io.wavfile as wav
from python_speech_features import mfcc

import os
import pickle 
import operator

In [3]:
def distance(instance1 , instance2 , k ):
    distance =0 
    mm1 = instance1[0] 
    cm1 = instance1[1]
    mm2 = instance2[0]
    cm2 = instance2[1]
    distance = np.trace(np.dot(np.linalg.inv(cm2), cm1)) 
    distance+=(np.dot(np.dot((mm2-mm1).transpose() , np.linalg.inv(cm2)) , mm2-mm1 )) 
    distance+= np.log(np.linalg.det(cm2)) - np.log(np.linalg.det(cm1))
    distance-= k
    return distance

In [4]:
 def getNeighbors(trainingSet , instance , k):
    distances =[]
    for x in range (len(trainingSet)):
        dist = distance(trainingSet[x], instance, k )+ distance(instance, trainingSet[x], k)
        distances.append((trainingSet[x][2], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    return neighbors   


In [5]:
def nearestClass(neighbors):
    classVote = {}

    for x in range(len(neighbors)):
        response = neighbors[x]
        if response in classVote:
            classVote[response]+=1 
        else:
            classVote[response]=1

    sorter = sorted(classVote.items(), key = operator.itemgetter(1), reverse=True)
    return sorter[0][0]

In [6]:
directory ="C:/Externship/Flask/genres/archive (2)/Data/genres_original/"
f = open("my.dat" ,'wb')
i = 0

In [7]:
for folder in os.listdir(directory):
    #as we have 10 classes, we're starting the loop from 1 to 11
    #so that we run the loop for total of 10 times, with each folder change (genre change),  i (label) changes. 
    i += 1
    if i==11 :
        break 
    print("fo",folder)
    for file in os.listdir(directory+folder):  
        
        #To read an Wav audio File in Python
        (rate,sig) = wav.read(directory+folder+"/"+file)
#         if directory+folder == "C:/Externship/Flask/genres/archive (4)/Data/genres_original/jazz":
#             print(directory+folder+"/"+file)
        #MFCC is the feature we will use for our analysis,
        #because it provides data about the overall shape of the audio frequencies.
        mfcc_feat = mfcc(sig,rate ,winlen=0.020, appendEnergy = False)
        covariance = np.cov(np.matrix.transpose(mfcc_feat))
        mean_matrix = mfcc_feat.mean(0)
        #making a feature typle that contains the mean matrix from mfcc as well as covariance,
        #and last variable in the feature tuple is the label (where numbers correspond to particular genre)
        feature = (mean_matrix , covariance , i)
        
        #This the the created dataset, which takes the input from specified path
        #it then stores the feature as a .dat file which can be used later without having to train all the files over it.
        pickle.dump(feature , f)
f.close()

fo blues
fo classical
fo country
fo disco
fo hiphop
fo jazz
fo metal
fo pop
fo reggae
fo rock


In [8]:
dataset = []
def loadDataset(project):
    with open("C:/Externship/Flask/my.dat" , 'rb') as f:
        while True:
            try:
                dataset.append(pickle.load(f))
            except EOFError:
                f.close()
                break  

loadDataset("C:/Externship/Flask/my.dat")

In [20]:
dataset = np.array(dataset)
dataset

[(array([ 76.50261107,  -1.96141736, -15.7743469 ,   3.8314189 ,
         -10.47332553,   1.31182833, -19.39373183,   5.28678994,
         -16.63172467,   5.3534445 ,  -9.85657026,   6.25007663,
          -5.58749505]),
  array([[ 3.13643266e+01, -1.34683421e+00, -9.48272203e-01,
          -2.05382281e+01, -1.45847814e+01, -1.72347943e+01,
          -9.35145739e+00,  2.72034545e+00, -3.29035455e+00,
           1.04057336e+00,  2.32513168e+00,  6.09119695e+00,
           8.44732834e+00],
         [-1.34683421e+00,  3.64751586e+01, -2.19100677e+01,
          -1.52506981e+01, -7.71247217e+00, -1.42477860e+01,
          -6.48177523e+00,  3.95463925e+00,  1.14509487e+00,
          -4.30576920e+00,  9.05817993e+00,  8.98145706e+00,
           7.68495192e+00],
         [-9.48272203e-01, -2.19100677e+01,  7.63578328e+01,
           1.69425823e+01, -2.67611115e+01, -2.71149039e+00,
           1.99049253e+01,  1.64116110e+01, -1.31355709e+01,
           1.53609455e+01, -1.31293962e+01, -7.478976

In [ ]:

from sklearn.model_selection import train_test_split
x_train ,x_test = train_test_split(dataset,test_size=0.15)


In [ ]:
leng = len(x_test)
predictions = []
for x in range (leng):
    predictions.append(nearestClass(getNeighbors(x_train ,x_test[x] , 8))) 

In [ ]:
def getAccuracy(testSet, predictions):
    #this is a variable to count total number of correct predictions.
    correct = 0 
    for x in range (len(testSet)):
        if testSet[x][-1]==predictions[x]:
            correct+=1
    return 1.0*correct/len(testSet)


In [1]:
accuracy1 = getAccuracy(x_test , predictions)
print(accuracy1*100)

NameError: name 'getAccuracy' is not defined